In [2]:
!mkdir -p rawdata && cd rawdata && wget https://synchronouscap.s3.eu-west-3.amazonaws.com/obchallenge/ETHUSDT-all.zip 

--2023-11-28 19:25:25--  https://synchronouscap.s3.eu-west-3.amazonaws.com/obchallenge/ETHUSDT-all.zip
Resolving synchronouscap.s3.eu-west-3.amazonaws.com (synchronouscap.s3.eu-west-3.amazonaws.com)... 16.12.18.22, 3.5.226.101
Connecting to synchronouscap.s3.eu-west-3.amazonaws.com (synchronouscap.s3.eu-west-3.amazonaws.com)|16.12.18.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8364338229 (7.8G) [application/zip]
Saving to: ‘ETHUSDT-all.zip.1’

ETHUSDT-all.zip.1     1%[                    ] 105.34M  14.0MB/s    eta 9m 12s ^C


In [13]:
!pwd

/home/sun/prog/ds/parse-ethusd/research


In [15]:
# list all files in archive
!unzip -l rawdata/ETHUSDT-all.zip

Archive:  rawdata/ETHUSDT-all.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
274554155  2023-01-11 15:59   ETHUSDT-2023-01-11.csv
498889432  2023-01-12 15:59   ETHUSDT-2023-01-12.csv
498940769  2023-01-13 15:59   ETHUSDT-2023-01-13.csv
494200797  2023-01-14 16:00   ETHUSDT-2023-01-14.csv
494686387  2023-01-15 15:59   ETHUSDT-2023-01-15.csv
494354267  2023-01-16 15:59   ETHUSDT-2023-01-16.csv
495055132  2023-01-17 15:59   ETHUSDT-2023-01-17.csv
495144362  2023-01-18 16:00   ETHUSDT-2023-01-18.csv
496322335  2023-01-19 15:59   ETHUSDT-2023-01-19.csv
496038324  2023-01-20 15:59   ETHUSDT-2023-01-20.csv
493911950  2023-01-21 15:59   ETHUSDT-2023-01-21.csv
494344998  2023-01-22 15:59   ETHUSDT-2023-01-22.csv
494359799  2023-01-23 15:59   ETHUSDT-2023-01-23.csv
494505851  2023-01-24 16:00   ETHUSDT-2023-01-24.csv
494711258  2023-01-25 15:59   ETHUSDT-2023-01-25.csv
494279336  2023-01-26 15:59   ETHUSDT-2023-01-26.csv
495525762  2023-01-27 15:59   ETHUSDT-2023-01-27

In [21]:
!mkdir -p splitted
# list all files             extract only file name            run bash shell for each file in parallel        unzip just selected file and compress it to gz
!time unzip -l rawdata/ETHUSDT-all.zip | egrep '([0-9A-Za-z\\-]+.csv)' -o | xargs -P 8 -I{} bash -c 'echo processing {} && unzip -p rawdata/ETHUSDT-all.zip {} | gzip > splitted/{}.gz'

processing ETHUSDT-2023-01-11.csv
processing ETHUSDT-2023-01-12.csv
processing ETHUSDT-2023-01-13.csv
processing ETHUSDT-2023-01-14.csv
processing ETHUSDT-2023-01-15.csv
processing ETHUSDT-2023-01-16.csv
processing ETHUSDT-2023-01-17.csv
processing ETHUSDT-2023-01-18.csv
processing ETHUSDT-2023-01-19.csv
processing ETHUSDT-2023-01-20.csv
processing ETHUSDT-2023-01-21.csv
processing ETHUSDT-2023-01-22.csv
processing ETHUSDT-2023-01-23.csv
processing ETHUSDT-2023-01-24.csv
processing ETHUSDT-2023-01-25.csv
processing ETHUSDT-2023-01-26.csv
processing ETHUSDT-2023-01-27.csv
processing ETHUSDT-2023-01-28.csv
processing ETHUSDT-2023-01-29.csv
processing ETHUSDT-2023-01-30.csv
processing ETHUSDT-2023-01-31.csv
processing ETHUSDT-2023-02-01.csv
processing ETHUSDT-2023-02-02.csv
processing ETHUSDT-2023-02-03.csv
processing ETHUSDT-2023-02-04.csv
processing ETHUSDT-2023-02-05.csv
processing ETHUSDT-2023-02-06.csv
processing ETHUSDT-2023-02-07.csv
processing ETHUSDT-2023-02-08.csv
processing ETH

In [22]:
# parse number of columns per line
!ls ./splitted/ | grep '\.csv\.gz'| time xargs -P 8 -I{} bash -c "zcat splitted/{} | python3 -c 'import sys; print(set(map(lambda row: len(row.split(\",\")), sys.stdin)))'" | tee > rows.txt

1053.92user 104.38system 2:31.99elapsed 762%CPU (0avgtext+0avgdata 9728maxresident)k
608inputs+0outputs (82major+1263519minor)pagefaults 0swaps


In [24]:
# check each row has same number of columns
!cat rows.txt | sort | uniq -c

    162 {4004}


In [25]:
# each file contains lines with exactly 4004 columns
#     0                        1-2             3-2002  2003-4002, 4003
# unix time in miliseconds, unknown costants, eth-usd, coef,    <empty>

In [31]:
# extract timestamps, some lines can have BOM char as prefix
!zcat splitted/*.csv.gz | time python3 -c  'import sys; list(map(lambda row: print(int(row.strip().strip("\ufeff")[:13])), sys.stdin))' > tm.txt

60.07user 28.26system 6:24.75elapsed 22%CPU (0avgtext+0avgdata 30660maxresident)k
0inputs+76072outputs (0major+14015minor)pagefaults 0swaps


In [32]:
!wc -l tm.txt

2777240 tm.txt


In [33]:
with open('tm.txt') as f:
    values = [int(v) for v in f]
len(values)

2777240

In [35]:
diffs = []
for i, val in enumerate(values):
    if i == 0:
        diffs.append(0)
        continue
    diffs.append(val - values[i-1])

diffs[:10]

[0, 5002, 5000, 5002, 5003, 5004, 5005, 5002, 4998, 5000]

In [37]:
import pandas as pd

In [38]:
df = pd.DataFrame(diffs[1:])
df.describe(percentiles=[i / 100 for i in range(1, 100)])

,0
count,2.777239e+06
mean,5.150187e+03
std,2.073819e+05
min,4.995000e+03
1%,5.004000e+03
...,...
96%,5.056000e+03
97%,5.062000e+03
98%,5.074000e+03
99%,5.102000e+03


In [40]:
# min distance in MS between frames, also no negative dist, data is ordered
st = list(sorted(diffs))
st[:10]

[0, 4995, 4995, 4995, 4995, 4996, 4996, 4996, 4996, 4996]

In [41]:
# max distance in MS between frames
st[-100:]

[5463,
 5469,
 5472,
 5473,
 5473,
 5474,
 5478,
 5478,
 5479,
 5485,
 5489,
 5490,
 5492,
 5493,
 5494,
 5503,
 5518,
 5519,
 5520,
 5521,
 5521,
 5523,
 5539,
 5546,
 5549,
 5552,
 5560,
 5563,
 5564,
 5566,
 5567,
 5570,
 5570,
 5584,
 5592,
 5593,
 5595,
 5601,
 5602,
 5605,
 5611,
 5611,
 5613,
 5633,
 5638,
 5638,
 5642,
 5658,
 5666,
 5705,
 5707,
 5707,
 5711,
 5719,
 5725,
 5744,
 5767,
 5801,
 5821,
 5828,
 5866,
 5868,
 5876,
 5878,
 5921,
 6008,
 6124,
 6213,
 6350,
 6441,
 8047,
 8314,
 8469,
 16256,
 19268,
 20542,
 26043,
 28450,
 30451,
 30492,
 30877,
 31275,
 33030,
 34372,
 36104,
 37018,
 39266,
 39371,
 39664,
 40041,
 40127,
 42500,
 45134,
 45940,
 48509,
 48986,
 55567,
 68633,
 90065,
 345607986]

In [43]:
!cd .. && cargo build --release

   Compiling autocfg v1.1.0
   Compiling libm v0.2.8
   Compiling cfg-if v1.0.0
   Compiling libc v0.2.150
   Compiling static_assertions v1.1.0
   Compiling semver v1.0.20
   Compiling version_check v0.9.4
   Compiling lexical-util v0.8.5      ] 0/118: static_assertions, versio...
   Compiling pkg-config v0.3.27       ] 1/118: static_assertions, versio...
   Compiling ahash v0.8.6             ] 4/118: version_check, autocfg, p...
   Compiling num-traits v0.2.17       ] 5/118: autocfg, ahash(build.rs),...
   Compiling num-integer v0.1.45      ] 6/118: num-traits(build.rs), aha...
   Compiling num-bigint v0.4.4        ] 7/118: num-traits(build.rs), aha...
   Compiling num-rational v0.4.1      ] 9/118: num-bigint(build.rs), num...
   Compiling num-iter v0.1.43        ] 10/118: num-bigint(build.rs), num...
   Compiling bytes v1.5.0            ] 17/118: num-bigint(build.rs), sem...    Building [==>                       ] 14/118: num-bigint(build.rs), lib...
   Compiling arrow-schema v49.0

In [47]:
!du -sch splitted/*

30M	splitted/ETHUSDT-2023-01-11.csv.gz
59M	splitted/ETHUSDT-2023-01-12.csv.gz
57M	splitted/ETHUSDT-2023-01-13.csv.gz
63M	splitted/ETHUSDT-2023-01-14.csv.gz
57M	splitted/ETHUSDT-2023-01-15.csv.gz
60M	splitted/ETHUSDT-2023-01-16.csv.gz
60M	splitted/ETHUSDT-2023-01-17.csv.gz
61M	splitted/ETHUSDT-2023-01-18.csv.gz
56M	splitted/ETHUSDT-2023-01-19.csv.gz
58M	splitted/ETHUSDT-2023-01-20.csv.gz
57M	splitted/ETHUSDT-2023-01-21.csv.gz
55M	splitted/ETHUSDT-2023-01-22.csv.gz
54M	splitted/ETHUSDT-2023-01-23.csv.gz
53M	splitted/ETHUSDT-2023-01-24.csv.gz
58M	splitted/ETHUSDT-2023-01-25.csv.gz
56M	splitted/ETHUSDT-2023-01-26.csv.gz
55M	splitted/ETHUSDT-2023-01-27.csv.gz
49M	splitted/ETHUSDT-2023-01-28.csv.gz
57M	splitted/ETHUSDT-2023-01-29.csv.gz
59M	splitted/ETHUSDT-2023-01-30.csv.gz
55M	splitted/ETHUSDT-2023-01-31.csv.gz
56M	splitted/ETHUSDT-2023-02-01.csv.gz
58M	splitted/ETHUSDT-2023-02-02.csv.gz
56M	splitted/ETHUSDT-2023-02-03.csv.gz
50M	splitted/ETHUSDT-2023-02-04.csv.gz
54M	splitted/ETHUSDT-2023

In [48]:
!

In [ ]:
!mkdir -p joined && ls splitted/ | grep '\.csv\.gz' | time xargs -P 8 -I{} bash -c 'echo processing {} && zcat splitted/{} | ../target/release/parse-ethusd > joined/{}.parquet'